<a href="https://colab.research.google.com/github/praveenprabharavindran/MachineLearning/blob/main/Build_Train_Deploy_Neural_Network_TensorFlow/House_Price_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries
* Numpy is a powerful n-dimensional array library that allows to easily create and manipulate arrays of data
* Numpy can convert TensofFlow's native data structures, to Python native data types
* Matplotlib is a graphics plot library

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

# Problem statement
 Using example data, develop a model that predicts house prices basecd on the size of a house

# Get Data
 In this example we will be using a truncated vesrion of the Ames dataset that only contains information on homes sold in May 2010.

## About the Ames Dataset
 The Ames dataset is a widely available dataset that has become one of thye standard datasets used when predicting home prices based on features of the home. It is based on the great work of Dean De Cock. His rational and insight into this dataset can be found at  https://jse.amstat.org/v19n3/decock.pdf.

 ## Getting the truncated dataset we use
 This dataset can be found with the excercies files for this course. The filename is AmesHousing-05-2010.csv

In [ ]:
from google.colab import files

# prompt  user to select a file to upload, and store it to a dictionary named 'uploaded' 
# 'uploded' is a dictionary with key = filename and value = contents of the file
uploaded = files.upload()

# 'iter()' function creates and iterator over the dictionary variable 'uploaded'
# 'next()' function iterates over the uploaded variable and gets the first file
# 'csv_housefile' variable is assigned the name of the file that was uploaded
csv_housefile = next(iter(uploaded))


print('User uploaded file: {name}, with lenght:{length}'.format(name=csv_housefile, length=len(uploaded[csv_housefile])))

# Load data into a pandas dataframe
* Pandas allows easil review and manipulation the data 
* Check out pandas webstie: https://pandas.pydata.org/
* Here is a [Pandas 10 minute intro](http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html) to understand how pandas, and pandas dataframes makes working with tabular data in Python easy.

In [ ]:
import pandas as pd
df_housing = pd.read_csv(csv_housefile)


# Visualize House Price Data

In [ ]:
pd.set_option('display.max_columns',None)
df_housing.head(5)

## Visualize Data using data_table formatter
* Using the data_table_formatter displays the table in an interactive mode making it easier to explore and analyze dataframes
* However this approach supports only 20 columns beyond which is fallsback to the pandas display.

In [ ]:
# from google.colab import data_table
# data_table.enable_dataframe_formatter()
# 
# columns = df_housing.columns
# display(df_housing[columns[:20]])


# Pre process data
* Among the given parameters (columns), the size of the house has the greatest impact on the cost. 
* The total square footage of the house is not immediately accessible and requires aggregating several fields together.  

 **Note**: There are some aggregate fields in the data that could be used to simply calucation:
  >Total Bsmt SF = BsmtFin SF 1 + BsmtFin SF 2 + Bsmt Unf SF  
  >Gr Liv Area = 1st Flr SF + 2nd Flr SF